In [3]:
%load_ext autoreload
%autoreload 2

# Setup Environment
import sys
import os
ROOT = '../../'
sys.path.append(ROOT)

from pettingzoo.classic import tictactoe_v3
import numpy as np
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from IPython.display import HTML
from source.agents.dqn_agent import DQNAgent
from source.agents.policy_gradient_with_baseline_agent import PolicyGradientWithBaselineAgent
from source.agents.random_agent import RandomAgent
from source.utils import utils
from tqdm import tqdm
import torch
import random
from typing import Dict, Optional, Tuple
from collections import defaultdict
from source.agents.agent import Agent
from pettingzoo.utils.env import AECEnv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Policy Eval
# Params
num_epoch = 50
num_episode = 100
video_path = os.path.join(ROOT, "video/dqn_tictactoe.mp4")
random_seed = 101

# Initialize
history = []
total_reward = 0
# set random seeds
rng = np.random.default_rng(random_seed)
#random.seed(random_seed)
np.random.seed(random_seed)
#torch.manual_seed(random_seed)
# epsilon decay
epsilon_schedule = utils.create_decay_schedule(num_epoch)
lr_schedule = utils.create_decay_schedule(num_epoch)

# Create Environment.
#env = gym.make('Taxi-v3')
# using render_mode=rgb_array so that video recording works
#env = gym.make(
#    "LunarLander-v2",
#    render_mode='rgb_array'
#)
#env = gym.make("CarRacing-v2", render_mode='rgb_array', continuous=False)
#env = gym.make("CartPole-v1", render_mode='rgb_array')

env = tictactoe_v3.env()
env.np_random = rng

# Create Agent
dqn_agent = DQNAgent(
    state_space=env.observation_space('player_1')['observation'],
    action_space=env.action_space('player_1'),
    discount_rate=0.99,
    epsilon=0.9, # use epsilon_schedule
    learning_rate=1e-3,
    learning=True,
    batch_size = 16,
    tau = 0.005,
    eps_decay=3000,
    net_params={'width':64, 'n_hidden':2},
    update_freq=100
)
pg_agent = PolicyGradientWithBaselineAgent(
    state_space=env.observation_space('player_1')['observation'],
    action_space=env.action_space('player_1'),
    discount_rate=0.99,
    epsilon=None, # use epsilon_schedule
    learning_rate=None,
    policy_lr=1e-3,
    value_lr=1e-3,
    net_params={'width':32, 'n_hidden':1}
)
random_agent = RandomAgent(    
    state_space=env.observation_space('player_2')['observation'],
    action_space=env.action_space('player_2'),
    discount_rate=None,
    epsilon=None, # use epsilon_schedule
    learning_rate=None,
    learning=False
)
agent_dict = {'player_1': dqn_agent, 'player_2':dqn_agent}
eps_history = []
# Start Learning
for i in range(num_epoch):
    success = 0
    for _ in tqdm(range(num_episode)):
        reward, steps = utils.play_multiagent_episode(agent_dict, env)#,epsilon=epsilon_schedule[i])
        history.append(reward['player_1'])
        eps_history.append(agent_dict['player_1']._epsilon)
        total_reward += reward['player_1']
    #score = agent.update()
    print(
        f"step: {i}, epsilon: {dqn_agent._epsilon}, average_return: {np.mean(history[-num_episode+1:-1])}, success rate: {success / num_episode}")
print(f"\nrewarding episodes: {total_reward}")

# For off policy learning only: get greedy policy (no exploration)
#agent._policy = get_epsilon_greedy_policy_from_action_values(agent._Q.weight)
# Run Eval
avarge_return, num_episode = utils.evaluate_multiagent(agent_dict, env, 5000)
for ag_id, value in avarge_return.items():
    print(f"{ag_id}: Average return = {value / num_episode}")

utils.plot_history(history)
utils.plot_history(eps_history, smoothing=False)

using device: cpu
using device: cpu


100%|██████████| 100/100 [00:07<00:00, 14.14it/s]


step: 0, epsilon: 0.050000000000010265, average_return: 0.23469387755102042, success rate: 0.0


100%|██████████| 100/100 [00:08<00:00, 12.36it/s]


step: 1, epsilon: 0.05000000000000001, average_return: 0.2857142857142857, success rate: 0.0


...

step: 2, epsilon: 0.05000000000000001, average_return: 0.16326530612244897, success rate: 0.0


100%|██████████| 100/100 [00:07<00:00, 13.81it/s]


step: 3, epsilon: 0.05000000000000001, average_return: 0.22448979591836735, success rate: 0.0


 32%|███▏      | 32/100 [00:02<00:04, 15.12it/s]


KeyboardInterrupt: 

In [ ]:
env = tictactoe_v3.env(render_mode='human')
reward, _ = utils.play_multiagent_episode(agent_dict, env)
#html=utils.render_mp4(video_path)
#HTML(html)


     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  -  |  O  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  -  |  O  |  -  
_____|_____|_____
     |     |     
  -  |  X  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  -  |  O  |  -  
_____|_____|_____
     |     |     
  O  |  X  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  X  |  O  |  -  
_____|_____|_____
     |     |     
  O  |  X  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     
     |     |     
  X  |  O  |  -  
_____|_____|_____
     |     |     
  O  |  X  |  X  
_____|_____|_____
     |     |     
  -  |  -  |  O  
     |     |     
     |     |     
  X  |  O 